In [1]:
!pip -q uninstall -y plotly
!pip -q install plotly==5.22.0 pandas numpy pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 76.9 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()
print("Uploaded:", list(uploaded.keys()))

Saving df_test.csv to df_test.csv
Saving df_test.parquet to df_test.parquet
Uploaded: ['df_test.csv', 'df_test.parquet']


In [4]:
import os
import numpy as np
import pandas as pd

print("Files:", os.listdir("."))

if os.path.exists("df_test.parquet"):
    df = pd.read_parquet("df_test.parquet")
    src = "df_test.parquet"
else:
    df = pd.read_csv("df_test.csv", index_col=0, parse_dates=True)
    src = "df_test.csv"

if not isinstance(df.index, pd.DatetimeIndex):
    df.index = pd.to_datetime(df.index, errors="coerce")

df = df.sort_index()
df = df[~df.index.isna()]

needed = ["p_crisis", "crisis_true", "ssi_true_fwd", "ssi_pred_fwd"]
missing = [c for c in needed if c not in df.columns]
assert not missing, f"Missing columns: {missing} | found: {list(df.columns)}"

for c in needed:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df["crisis_true"] = df["crisis_true"].fillna(0).astype(int)
df["p_crisis"] = df["p_crisis"].clip(0, 1)

df = df.dropna(subset=["p_crisis", "ssi_pred_fwd"])

print("Source:", src, "| Shape:", df.shape)
print("Date range:", df.index.min(), "->", df.index.max())
display(df.head())

Files: ['.config', 'df_test.csv', 'df_test.parquet', 'sample_data']
Source: df_test.parquet | Shape: (626, 4)
Date range: 2023-04-18 00:00:00 -> 2025-11-12 00:00:00


,p_crisis,crisis_true,ssi_true_fwd,ssi_pred_fwd
2023-04-18,0.009749,0,0.869352,0.781657
2023-04-19,0.009710,0,0.757848,0.777657
2023-04-20,0.009743,0,0.750847,0.773666
2023-04-21,0.009865,0,0.634849,0.768359
2023-04-24,0.010785,0,0.608512,0.752664


In [5]:
# --- SSI percentile thresholds (global) ---
q50 = float(df["ssi_pred_fwd"].quantile(0.50))
q80 = float(df["ssi_pred_fwd"].quantile(0.80))
q95 = float(df["ssi_pred_fwd"].quantile(0.95))

def band_from_ssi(v):
    if v < q50:  return "GREEN"
    if v < q80:  return "YELLOW"
    if v < q95:  return "ORANGE"
    return "RED"

df["band"] = df["ssi_pred_fwd"].apply(band_from_ssi)

# --- Alarm rule (interpretable default) ---
ssi_thr = float(df["ssi_pred_fwd"].quantile(0.90))
p_thr = 0.20
df["alarm"] = ((df["ssi_pred_fwd"] >= ssi_thr) | (df["p_crisis"] >= p_thr)).astype(int)

# --- Rolling summaries for last 30 / 90 days (calendar days) ---
last_date = df.index.max()
def window_slice(days):
    start = last_date - pd.Timedelta(days=days)
    return df.loc[df.index >= start].copy()

df30 = window_slice(30)
df90 = window_slice(90)

summary = {
    "30d_alarm_cnt": int(df30["alarm"].sum()) if len(df30) else 0,
    "30d_max_ssi": float(df30["ssi_pred_fwd"].max()) if len(df30) else np.nan,
    "30d_max_p": float(df30["p_crisis"].max()) if len(df30) else np.nan,

    "90d_alarm_cnt": int(df90["alarm"].sum()) if len(df90) else 0,
    "90d_max_ssi": float(df90["ssi_pred_fwd"].max()) if len(df90) else np.nan,
    "90d_max_p": float(df90["p_crisis"].max()) if len(df90) else np.nan,
}

latest = df.iloc[-1]
print("Latest:", df.index[-1].date(),
      "| band:", latest["band"],
      "| SSI:", float(latest["ssi_pred_fwd"]),
      "| p:", float(latest["p_crisis"]),
      "| alarm:", int(latest["alarm"]))
print("Summary:", summary)

Latest: 2025-11-12 | band: GREEN | SSI: 0.04760979488492012 | p: 0.030669622123241425 | alarm: 0
Summary: {'30d_alarm_cnt': 0, '30d_max_ssi': 0.1817241907119751, '30d_max_p': 0.043758541345596313, '90d_alarm_cnt': 9, '90d_max_ssi': 0.5960350036621094, '90d_max_p': 0.35350683331489563}


In [6]:
import plotly.graph_objects as go
from plotly.offline import plot
from datetime import datetime

# -----------------------------
# Helper: build colored regime stripes as plotly shapes
# -----------------------------
band_color = {
    "GREEN":  "rgba(46, 204, 113, 0.12)",
    "YELLOW": "rgba(241, 196, 15, 0.12)",
    "ORANGE": "rgba(230, 126, 34, 0.12)",
    "RED":    "rgba(231, 76, 60, 0.12)",
}

# Create segments where band is constant
bands = df["band"].values
dates = df.index.to_pydatetime()

segments = []
seg_start = dates[0]
seg_band = bands[0]

for i in range(1, len(df)):
    if bands[i] != seg_band:
        seg_end = dates[i-1]
        segments.append((seg_start, seg_end, seg_band))
        seg_start = dates[i]
        seg_band = bands[i]
segments.append((seg_start, dates[-1], seg_band))

shapes = []
for x0, x1, b in segments:
    shapes.append(dict(
        type="rect",
        xref="x", yref="paper",
        x0=x0, x1=x1,
        y0=0, y1=1,
        fillcolor=band_color[b],
        line=dict(width=0),
        layer="below"
    ))

# -----------------------------
# Build main chart
# -----------------------------
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.index, y=df["ssi_pred_fwd"],
    name="SSI forecast (t+H)",
    mode="lines"
))

fig.add_trace(go.Scatter(
    x=df.index, y=df["p_crisis"],
    name="Crisis probability",
    mode="lines"
))

# crisis markers
cr_dates = df.index[df["crisis_true"] == 1]
if len(cr_dates) > 0:
    fig.add_trace(go.Scatter(
        x=cr_dates,
        y=df.loc[cr_dates, "p_crisis"],
        name="True crisis label",
        mode="markers",
        marker=dict(size=9, symbol="x")
    ))

# alarm markers
al_dates = df.index[df["alarm"] == 1]
if len(al_dates) > 0:
    fig.add_trace(go.Scatter(
        x=al_dates,
        y=df.loc[al_dates, "p_crisis"],
        name="Alarm",
        mode="markers",
        marker=dict(size=8, symbol="triangle-up")
    ))

fig.update_layout(
    title=dict(text="EconSSI — Offline Systemic Stress Report", x=0.5),
    height=600,
    xaxis_title="Date",
    yaxis_title="Value (p in 0–1; SSI in 0–1 demo scale)",
    legend=dict(orientation="h", y=1.06),
    shapes=shapes
)

plot_div = plot(fig, output_type="div", include_plotlyjs="cdn")

# -----------------------------
# Mini summary cards HTML
# -----------------------------
latest = df.iloc[-1]
latest_date = df.index[-1].date()

def fmt(x):
    return "—" if (x is None or (isinstance(x, float) and np.isnan(x))) else f"{x:.3f}"

kpi_html = f"""
<div style="font-family: Arial, sans-serif; margin: 8px 0 18px 0;">
  <h2 style="margin: 0;">EconSSI — Offline Report</h2>
  <div style="color:#444; margin-top:6px;">
    Generated: {datetime.utcnow().strftime('%Y-%m-%d %H:%M UTC')} | Data range: {df.index.min().date()} → {df.index.max().date()}
  </div>

  <div style="display:flex; gap:12px; margin-top:14px; flex-wrap:wrap;">
    <div style="padding:12px 14px; border:1px solid #ddd; border-radius:10px; min-width:220px;">
      <div style="color:#666; font-size:12px;">Latest date</div>
      <div style="font-size:18px; font-weight:700;">{latest_date}</div>
    </div>
    <div style="padding:12px 14px; border:1px solid #ddd; border-radius:10px; min-width:220px;">
      <div style="color:#666; font-size:12px;">Latest SSI(t+H)</div>
      <div style="font-size:18px; font-weight:700;">{float(latest['ssi_pred_fwd']):.3f}</div>
    </div>
    <div style="padding:12px 14px; border:1px solid #ddd; border-radius:10px; min-width:220px;">
      <div style="color:#666; font-size:12px;">Latest crisis probability</div>
      <div style="font-size:18px; font-weight:700;">{float(latest['p_crisis']):.3f}</div>
    </div>
    <div style="padding:12px 14px; border:1px solid #ddd; border-radius:10px; min-width:220px;">
      <div style="color:#666; font-size:12px;">Traffic-light band</div>
      <div style="font-size:18px; font-weight:700;">{latest['band']}</div>
      <div style="color:#666; font-size:11px; margin-top:6px;">
        GREEN&lt;p50({q50:.3f}) · YELLOW&lt;p80({q80:.3f}) · ORANGE&lt;p95({q95:.3f}) · RED≥p95
      </div>
    </div>
  </div>

  <h3 style="margin:18px 0 8px 0;">Quick summaries</h3>
  <div style="display:flex; gap:12px; flex-wrap:wrap;">
    <div style="padding:12px 14px; border:1px solid #ddd; border-radius:10px; min-width:320px;">
      <div style="font-weight:700; margin-bottom:6px;">Last 30 days</div>
      <div style="color:#333;">Alarm count: <b>{summary['30d_alarm_cnt']}</b></div>
      <div style="color:#333;">Max SSI: <b>{fmt(summary['30d_max_ssi'])}</b></div>
      <div style="color:#333;">Max p(crisis): <b>{fmt(summary['30d_max_p'])}</b></div>
    </div>
    <div style="padding:12px 14px; border:1px solid #ddd; border-radius:10px; min-width:320px;">
      <div style="font-weight:700; margin-bottom:6px;">Last 90 days</div>
      <div style="color:#333;">Alarm count: <b>{summary['90d_alarm_cnt']}</b></div>
      <div style="color:#333;">Max SSI: <b>{fmt(summary['90d_max_ssi'])}</b></div>
      <div style="color:#333;">Max p(crisis): <b>{fmt(summary['90d_max_p'])}</b></div>
    </div>
    <div style="padding:12px 14px; border:1px solid #ddd; border-radius:10px; min-width:320px;">
      <div style="font-weight:700; margin-bottom:6px;">Alarm rule (interpretable)</div>
      <div style="color:#333;">Alarm if <b>SSI ≥ p90 ({ssi_thr:.3f})</b> OR <b>p(crisis) ≥ {p_thr:.2f}</b></div>
      <div style="color:#666; font-size:11px; margin-top:6px;">
        This is a monitoring trigger, not a trading signal.
      </div>
    </div>
  </div>
</div>
"""

# Top alarm days table
top_tbl = df[df["alarm"] == 1].copy()
top_tbl["date"] = top_tbl.index.date
top_tbl = top_tbl.sort_values(["p_crisis", "ssi_pred_fwd"], ascending=False).head(20)
top_tbl = top_tbl[["date", "p_crisis", "ssi_pred_fwd", "crisis_true", "band"]]
table_html = top_tbl.to_html(index=False, float_format=lambda x: f"{x:.4f}")

notes_html = """
<div style="font-family: Arial, sans-serif; margin-top: 14px; color:#333;">
  <h3>How to interpret (simple)</h3>
  <ul>
    <li><b>Background colors</b> show stress regimes (GREEN→RED) based on SSI percentiles.</li>
    <li><b>SSI</b> tracks regime-level build-up of fragility (not guaranteed crash timing).</li>
    <li><b>p(crisis)</b> is event-level and can remain low under rare-event dynamics.</li>
  </ul>
  <div style="font-size:12px; color:#666;">
    Research demo only. No financial advice. Uses public market data / derived signals only.
  </div>
</div>
"""

html = f"""
<!doctype html>
<html>
<head>
  <meta charset="utf-8"/>
  <title>EconSSI Offline Report</title>
</head>
<body>
  {kpi_html}
  {plot_div}
  <div style="font-family: Arial, sans-serif; margin-top: 10px;">
    <h3>Top alarm days (Top 20)</h3>
    {table_html}
  </div>
  {notes_html}
</body>
</html>
"""

out_path = "econssi_report.html"
with open(out_path, "w", encoding="utf-8") as f:
    f.write(html)

print("Saved:", out_path)

Saved: econssi_report.html


/tmp/ipython-input-154597870.py:106: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



In [7]:
from google.colab import files
files.download("econssi_report.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>